# Aggrégation des fichiers de data.gouv.fr pour les PLF, PLR, Recettes - Introduction

In [50]:
import csv
import json
import os
import re

from tqdm import tqdm

# Décodage des fichiers
Réécrire les fichier dans un encoding compatible `UTF-8`.

In [64]:
RAW_FOLDER = './input_data/raw_data/'

for folderName in os.listdir(RAW_FOLDER):
    DECODED_FOLDER = './input_data/decoded_data/{}'.format(folderName)

    if not os.path.exists(DECODED_FOLDER):
        os.makedirs(DECODED_FOLDER)

    currentPath = os.path.join(RAW_FOLDER, folderName)
    for fileName in os.listdir(currentPath):
        os.system('iconv -f WINDOWS-1252 -t UTF-8 {} > {}'.format(os.path.join(currentPath, fileName), os.path.join(DECODED_FOLDER, fileName)))
#         os.system('iconv -f ISO-8859-1 -t UTF-8 {} > {}'.format(os.path.join(currentPath, fileName), os.path.join(DECODED_FOLDER, fileName)))

# Conversion des nomenclatures en hiérarchies D3

## Utilitaires

Une fonction pour calculer un attribut d'un noeud récursivement.

In [65]:
def recKey(node, key, default_value=0):
    # Si le noeud n'est pas une feuille, on calcule sa valeur récursivement
    if key not in node:
        node[key] = sum(list(map(lambda x: recKey(x, key), node['children'])))

    return node[key] if node[key] else default_value

`build_flat_tree` prend un dictionnaire `d`, un arbre `node` et une clé `key`.

Elle remplit `d` en parcourant `node` récursivement, afin de construire une représentation plate de `node` triée avec la clé `key`.

In [66]:
def build_flat_tree(d, node, key='code'):
    rnode = node.copy()

    if 'children' in rnode:
        del rnode['children']

    d[node[key]] = rnode

    if 'children' in node:
        for child in node['children']:
            build_flat_tree(d, child, key)

`build_d3_hierarchy` parcourt un fichier CSV contenant les feuilles d'un arbre et construit une hiérarchie D3 représentant cet arbre.

In [67]:
def build_d3_hierarchy(rows, root_node, text_indices, code_indices, modify_current_node=None, callback=None):
    for row in rows:
        current_node = root_node

        nodes = [field for field in [row[i] for i in text_indices] if field != ''] 
        for node_index, node in enumerate(nodes):
            # On cherche si le chemin existe déjà.
            # S'il n'existe pas, on le crée.
            found = False
            if not 'children' in current_node:
                current_node['children'] = []
            for index, child in enumerate(current_node['children']):
                if child['name'] == node:
                    current_node = current_node['children'][index]
                    found = True
                    break

            if not found:
                current_node['children'].append(dict({
                    'name': node,
                    'code': '-'.join([row[code_indices[i]] for i in range(node_index+1)])
                }))
                current_node = current_node['children'][-1]

                # Si le noeud n'est pas une feuille, il a des enfants...
                if node_index != len(nodes)-1:
                    current_node['children'] = []
                else:
                    if (modify_current_node is not None):
                        modify_current_node(current_node, row)

    # On calcule des cp et les ae des noeuds qui ne
    # sont pas des feuilles.
    if callback is not None:
        callback(root_node)
        
    return root_node

Deux fonctions pour exporter les résultats calculés.

In [68]:
def write_d3_hierarchy(root_node, target_file_name):
    with open(target_file_name, 'w+') as output_file:
        json.dump(root_node, output_file)

def write_flat_hierarchy(root_node, target_flat_file_name):
    with open(target_flat_file_name, 'w+') as output_file:
        d = dict()
        build_flat_tree(d, root_node)

        json.dump(d, output_file)

## Tests

In [29]:
init_node = dict({
    'name': 'PLF',
    'children': [],
    'code': 'PLF'
})

with open('./input_data/nomenclatures_plf/plf_2019.csv', 'r') as in_file:
    csv_reader = csv.reader(in_file, delimiter=',')
    rows = [row for row in csv_reader]

root_node = build_d3_hierarchy(
    rows, init_node,                        
    [2, 4, 6, 8, 10], [1, 3, 5, 7, 9]
)

## Création des hiérarchies

In [30]:
type_de_mission_to_code = {
    "Budget général": "1",
    "Budgets annexes": "2",
    "Comptes d'affectation spéciale": "3",
    "Comptes de concours financiers": "4"
}

### 2019

#### PLF

In [31]:
SOURCE_FOLDER = './input_data/decoded_data/plf_2019'

type_de_mission_to_filename = {
    "Budget général": "bg",
    "Budgets annexes": "ba",
    "Comptes d'affectation spéciale": "cs",
    "Comptes de concours financiers": "cf"
}

In [32]:
mission_to_code = {}

with open(os.path.join(SOURCE_FOLDER, 'plf2019-nomenclature-mpa.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=';')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[2]
        mission = row[3]

        mission_to_code[mission] = code_mission

On empile les fichiers qui nous intéressent (ici, la comptabilité générale du BG, CAS CCF et BA).

In [33]:
rows = []
credits_paiement_total = 0
autorisations_engagement_total = 0

for type_de_mission in type_de_mission_to_code:
    code_type_de_mission = type_de_mission_to_code[type_de_mission]
    code_filename = type_de_mission_to_filename[type_de_mission]

    with open(os.path.join(
        SOURCE_FOLDER,
        'plf2019-{}-msn-dest.csv'.format(code_filename)), 'r') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=';')
        # On saute la première ligne parce que c'est un header
        next(csv_reader, None)

        for index, row in enumerate(csv_reader):
            # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
            if (row[2] == ''):
                continue

            autorisations_engagement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
            autorisations_engagement_total += autorisations_engagement
            credits_paiement = int(row[10].replace(' ', '')) if row[10].replace(' ', '') != '' else 0
            credits_paiement_total += credits_paiement

            code_action = '0' + row[5] if len(row[5]) == 1 else row[5]
            code_sousaction = '0' + row[7] if len(row[7]) == 1 else row[7]

            # On écrit dans le fichier cible toutes les informations utiles
            rows.append([
                type_de_mission,
                code_type_de_mission,
                row[2],
                mission_to_code[row[2]],
                row[4],
                row[3],
                row[6],
                code_action,
                row[8],
                code_sousaction,
                autorisations_engagement,
                credits_paiement
            ])

print(credits_paiement_total, autorisations_engagement_total)

675353577993 689624049061


On construit la hiéarchie D3.

In [34]:
init_node = dict({
    'name': 'PLF',
    'children': [],
    'code': 'PLF'
})

def modify_current_node(current_node, row):
    current_node['cp'] = int(row[11])
    current_node['ae'] = int(row[10])
    current_node['size'] = int(row[11])

def callback(root_node):
    recKey(root_node, 'cp')
    recKey(root_node, 'ae')
    recKey(root_node, 'size')

root_node = build_d3_hierarchy(
    rows, init_node,
    [0, 2, 4, 6, 8], [1, 3, 5, 7, 9],
    modify_current_node=modify_current_node,
    callback=callback
)

write_d3_hierarchy(root_node, '../server/resources/hierarchy/plf/2019.json')
write_flat_hierarchy(root_node, '../server/resources/flat/plf/2019.json')

#### Recettes

In [71]:
with open('./input_data/decoded_data/plf_2019/plf2019-recettes-fiscales-etata.csv', 'r', encoding='utf-8') as recettes:
    csv_reader = csv.reader(recettes, delimiter=';')
    next(csv_reader)

    root_node = dict({
        'name': 'Recettes',
        'children': [],
        'code': 'REC'
    })

    for row in csv_reader:
        line_number = row[0]
        label = row[1]
        montant = int(row[2].replace(" ", '')) if row[2] else None

        if line_number:
            codes = [line_number[0], line_number[1], line_number[2:]]
        else:
            m = re.match('(.*)\.\s?(.*)', label)
            codes = list(m.group(1))
            label = m.group(2)

        current_node = root_node

        for code_index, code in enumerate(codes[:-1]):
            if 'children' not in current_node:
                current_node['children'] = []

            for index, child in enumerate(current_node['children']):
                if child['code'].startswith('-'.join(codes[:code_index+1])):
                    current_node = current_node['children'][index]
        
        if 'children' not in current_node:
            current_node['children'] = []

        current_node['children'].append(dict({
            'name': label,
            'code': '-'.join(codes),
        }))

        # Terrible workaround for finding tree leaves
        if len(codes) == 3 or codes == ['4', '1']:
            current_node['children'][-1]['size'] = montant

    recKey(root_node, 'size')

write_d3_hierarchy(root_node, '../server/resources/hierarchy/recettes/2019.json')
write_flat_hierarchy(root_node, '../server/resources/flat/recettes/2019.json')

### 2018

In [37]:
SOURCE_FOLDER = './input_data/decoded_data/plf_2018'

type_de_mission_to_filename = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
    "Comptes de concours financiers": "CF"
}

In [38]:
mission_to_code = {}
code_to_programme = {}

with open(os.path.join('./input_data/nomenclatures_plf/plf_2018.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme

In [39]:
rows = []
credits_paiement_total = 0
autorisations_engagement_total = 0

for type_de_mission in type_de_mission_to_code:
    code_type_de_mission = type_de_mission_to_code[type_de_mission]
    code_filename = type_de_mission_to_filename[type_de_mission]

    with open(os.path.join(
        SOURCE_FOLDER,
        'PLF2018-{}-Msn-Dest.csv'.format(code_filename)), 'r') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=';')
        # On saute les headers (4 lignes)
        for i in range(4):
            next(csv_reader, None)

        for index, row in enumerate(csv_reader):
            # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
            if (row[2] == ''):
                continue

            autorisations_engagement = int(row[7].replace(' ', '')) if row[7].replace(' ', '') != '' else 0
            autorisations_engagement_total += autorisations_engagement
            credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
            credits_paiement_total += credits_paiement

            # On écrit dans le fichier cible toutes les informations utiles
            rows.append([
                type_de_mission,
                code_type_de_mission,
                row[0],
                mission_to_code[row[0]],
                code_to_programme[row[1]] if row[1] in code_to_programme else 'Nom de programme introuvable',
                row[1],
                row[3],
                row[2],
                row[5],
                row[4],
                autorisations_engagement,
                credits_paiement
            ])

print(credits_paiement_total, autorisations_engagement_total)

648238506383 651922195355


In [40]:
init_node = dict({
    'name': 'PLF',
    'children': [],
    'code': 'PLF'
})

def modify_current_node(current_node, row):
    current_node['cp'] = int(row[11])
    current_node['ae'] = int(row[10])
    current_node['size'] = int(row[11])

def callback(root_node):
    recKey(root_node, 'cp')
    recKey(root_node, 'ae')
    recKey(root_node, 'size')

root_node = build_d3_hierarchy(
    rows, init_node,
    [0, 2, 4, 6, 8], [1, 3, 5, 7, 9],
    modify_current_node=modify_current_node,
    callback=callback
)

write_d3_hierarchy(root_node, '../server/resources/hierarchy/plf/2018.json')
write_flat_hierarchy(root_node, '../server/resources/flat/plf/2018.json')

#### Recettes

In [70]:
with open('./input_data/decoded_data/plf_2018/PLF2018-Recettes_fiscales-EtatA.csv', 'r') as recettes:
    csv_reader = csv.reader(recettes, delimiter=';')
    for _ in range(4):
        next(csv_reader)

    root_node = dict({
        'name': 'Recettes',
        'children': [],
        'code': 'REC'
    })

    for row in csv_reader:
        line_number = row[0]
        label = row[1]
        montant = int(row[2].replace(" ", '')) if row[2] else None

        if line_number:
            codes = [line_number[0], line_number[1], line_number[2:]]
        else:
            m = re.match('(.*)\.\s?(.*)', label)
            codes = list(m.group(1))
            label = m.group(2)

        current_node = root_node
        for code_index, code in enumerate(codes[:-1]):
            if 'children' not in current_node:
                current_node['children'] = []

            for index, child in enumerate(current_node['children']):
                if child['code'].startswith('-'.join(codes[:code_index+1])):
                    current_node = current_node['children'][index]

        if 'children' not in current_node:
            current_node['children'] = []

        current_node['children'].append(dict({
            'name': label,
            'code': '-'.join(codes),
        }))

        # Terrible workaround for finding tree leaves
        if len(codes) == 3 or codes == ['4', '1']:
            current_node['children'][-1]['size'] = montant

    recKey(root_node, 'size')

write_d3_hierarchy(root_node, '../server/resources/hierarchy/recettes/2018.json')
write_flat_hierarchy(root_node, '../server/resources/flat/recettes/2018.json')

### 2017

In [258]:
SOURCE_FOLDER = './input_data/decoded_data/plf_2017'

type_de_mission_to_filename = {
    "Budget général": "BG",
#     "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
    "Comptes de concours financiers": "CF"
}

In [260]:
mission_to_code = {}
code_to_programme = {}
code_to_action = {}

with open(os.path.join('./input_data/nomenclatures_plf/plf_2017.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]
        code_action = row[7]
        action = row[8]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme
        code_to_action['-'.join([code_programme, code_action])] = action

In [261]:
rows = []
credits_paiement_total = 0
autorisations_engagement_total = 0

for type_de_mission in type_de_mission_to_code:
    if type_de_mission == 'Budgets annexes':
        continue

    code_type_de_mission = type_de_mission_to_code[type_de_mission]
    code_filename = type_de_mission_to_filename[type_de_mission]

    with open(os.path.join(
        SOURCE_FOLDER,
        'PLF2017-{}-Msn-Nat.csv'.format(code_filename)), 'r') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=';')
        # On saute les headers (4 lignes)
        for i in range(4):
            next(csv_reader, None)

        for index, row in enumerate(csv_reader):
            autorisations_engagement = int(row[6].replace(' ', '')) if row[6].replace(' ', '') != '' else 0
            autorisations_engagement_total += autorisations_engagement
            credits_paiement = int(row[8].replace(' ', '')) if row[8].replace(' ', '') != '' else 0
            credits_paiement_total += credits_paiement

            code_action = '0' + row[2] if len(row[2]) == 1 else row[2]
            ccode_action = '-'.join([row[1], code_action])

            # On écrit dans le fichier cible toutes les informations utiles
            rows.append([
                type_de_mission,
                code_type_de_mission,
                row[0],
                mission_to_code[row[0]],
                code_to_programme[row[1]] if row[1] in code_to_programme else 'Nom de programme introuvable',
                row[1],
                code_to_action[ccode_action] if ccode_action in code_to_action else 'Nom d\'action introuvable',
                code_action,
                row[4],
                row[3],
                autorisations_engagement,
                credits_paiement
            ])

print(credits_paiement_total, autorisations_engagement_total)

630390141338 650044472748


In [270]:
init_node = dict({
    'name': 'PLF',
    'children': [],
    'code': 'PLF'
})

def modify_current_node(current_node, row):
    current_node['cp'] = int(row[11])
    current_node['ae'] = int(row[10])
    current_node['size'] = int(row[11])

def callback(root_node):
    recKey(root_node, 'cp')
    recKey(root_node, 'ae')
    recKey(root_node, 'size')

root_node = build_d3_hierarchy(
    rows, init_node,
    [0, 2, 4, 6, 8], [1, 3, 5, 7, 9],
    modify_current_node=modify_current_node,
    callback=callback
)

On a calculé le PLF 2017 avec les données de la comptabilité générale... Il faut couper l'arbre après le niveau `action`.

In [271]:
def rec_cut_tree(node, depth, max_depth=4):
    d = dict({
        'name': node['name'],
        'code': node['code'],
        'ae': node['ae'],
        'cp': node['cp'],
        'size': node['size']
    })

    if depth < max_depth:
        d['children'] = [rec_cut_tree(child, depth+1, max_depth=max_depth) for child in node['children']]

    return d

In [272]:
cut_tree = rec_cut_tree(root_node, 0, max_depth=4)
with open('../server/resources/hierarchy/plf/2017.json', 'w') as output_json:
    json.dump(cut_tree, output_json)

with open('../server/resources/flat/plf/2017.json', 'w+') as output_file:
    d = dict()
    build_flat_tree(d, cut_tree)

    json.dump(d, output_file)

### 2016

In [277]:
SOURCE_FOLDER = './input_data/decoded_data/plf_2016'

type_de_mission_to_filename = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
#     "Comptes de concours financiers": "CF"
}

In [279]:
mission_to_code = {}
code_to_programme = {}

with open(os.path.join('./input_data/nomenclatures_plf/plf_2016.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme

In [280]:
rows = []
credits_paiement_total = 0
autorisations_engagement_total = 0

for type_de_mission in type_de_mission_to_code:
     # CAS SPECIAL ICI : il n'y a pas de CF, il me semble qu'ils sont dans les CS...
    if type_de_mission == 'Comptes de concours financiers':
        continue
    code_type_de_mission = type_de_mission_to_code[type_de_mission]
    code_filename = type_de_mission_to_filename[type_de_mission]

    with open(os.path.join(
        SOURCE_FOLDER,
        'PLF2016-{}-Msn-Dest.csv'.format(code_filename)), 'r') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=';')
        # On saute les headers (4 lignes)
        for i in range(4):
            next(csv_reader, None)

        for index, row in enumerate(csv_reader):
            # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
            if (row[2] == ''):
                continue

            autorisations_engagement = int(row[7].replace(' ', '')) if row[7].replace(' ', '') != '' else 0
            autorisations_engagement_total += autorisations_engagement
            credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
            credits_paiement_total += credits_paiement

            # On écrit dans le fichier cible toutes les informations utiles
            rows.append([
                type_de_mission,
                code_type_de_mission,
                row[0],
                mission_to_code[row[0]],
                code_to_programme[row[1]] if row[1] in code_to_programme else 'Nom de programme introuvable',
                row[1],
                row[3],
                row[2],
                row[5],
                row[4],
                autorisations_engagement,
                credits_paiement
            ])

print(credits_paiement_total, autorisations_engagement_total)

744156246731 744156246731


In [281]:
init_node = dict({
    'name': 'PLF',
    'children': [],
    'code': 'PLF'
})

def modify_current_node(current_node, row):
    current_node['cp'] = int(row[11])
    current_node['ae'] = int(row[10])
    current_node['size'] = int(row[11])

def callback(root_node):
    recKey(root_node, 'cp')
    recKey(root_node, 'ae')
    recKey(root_node, 'size')

root_node = build_d3_hierarchy(
    rows, init_node,
    [0, 2, 4, 6, 8], [1, 3, 5, 7, 9],
    modify_current_node=modify_current_node,
    callback=callback
)

write_d3_hierarchy(root_node, '../server/resources/hierarchy/plf/2016.json')
write_flat_hierarchy(root_node, '../server/resources/flat/plf/2016.json')

### 2015

In [282]:
SOURCE_FOLDER = './input_data/decoded_data/plf_2015'

type_de_mission_to_filename = {
    "Budget général": "BG",
    "Budgets annexes": "BA",
    "Comptes d'affectation spéciale": "CS",
#     "Comptes de concours financiers": "CF"
}

In [283]:
mission_to_code = {}
code_to_programme = {}

with open(os.path.join('./input_data/nomenclatures_plf/plf_2015.csv'), 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    # On saute la première ligne parce que c'est un header
    next(csv_reader, None)

    for row in csv_reader:
        code_mission = row[3]
        mission = row[4]
        code_programme = row[5]
        programme = row[6]

        mission_to_code[mission] = code_mission
        code_to_programme[code_programme] = programme

In [284]:
rows = []
credits_paiement_total = 0
autorisations_engagement_total = 0

for type_de_mission in type_de_mission_to_code:
     # CAS SPECIAL ICI : il n'y a pas de CF, il me semble qu'ils sont dans les CS...
    if type_de_mission == 'Comptes de concours financiers':
        continue
    code_type_de_mission = type_de_mission_to_code[type_de_mission]
    code_filename = type_de_mission_to_filename[type_de_mission]

    with open(os.path.join(
        SOURCE_FOLDER,
        'PLF2015-{}-Msn-Dest.csv'.format(code_filename)), 'r') as csv_file:

        csv_reader = csv.reader(csv_file, delimiter=';')
        # On saute les headers (4 lignes)
        for i in range(4):
            next(csv_reader, None)

        for index, row in enumerate(csv_reader):
            # Petite subtilité pour le cas 2019, certains documents csv ont trop de lignes (il y a des lignes vides à la fin)
            if (row[2] == ''):
                continue

            autorisations_engagement = int(row[7].replace(' ', '')) if row[7].replace(' ', '') != '' else 0
            autorisations_engagement_total += autorisations_engagement
            credits_paiement = int(row[9].replace(' ', '')) if row[9].replace(' ', '') != '' else 0
            credits_paiement_total += credits_paiement

            # On écrit dans le fichier cible toutes les informations utiles
            rows.append([
                type_de_mission,
                code_type_de_mission,
                row[0],
                mission_to_code[row[0]],
                code_to_programme[row[1]] if row[1] in code_to_programme else 'Nom de programme introuvable',
                row[1],
                row[3],
                row[2],
                row[5],
                row[4],
                autorisations_engagement,
                credits_paiement
            ])

print(credits_paiement_total, autorisations_engagement_total)

KeyError: "Gestion et valorisation des ressources tirées de l'utilisation du spectre hertzien, des systèmes et des infrastructures de télécommunications de l'État"

```Toute chose appartient à qui sait en jouir.```